In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import json
import plotly.express as px
import plotly.io as pio

pio.renderers.default = 'browser'

india_states=json.load(open("states_india.geojson","r"))
Data_Frame=pd.read_csv('lane-accidents.csv')


state_id_map={}
for feature in india_states['features']:
    feature['id']=feature['properties']['state_code']
    state_id_map[feature['properties']['st_nm']]= feature['id']

Data_Frame.at[33,'State/UT']='NCT of Delhi'
Data_Frame.at[1,'State/UT']='Arunanchal Pradesh'
Data_Frame.at[19,'State/UT']='Odisha'
Data_Frame.at[29,'State/UT']='Andaman & Nicobar Island'
Data_Frame.at[31,'State/UT']='Dadara & Nagar Havelli'
Data_Frame=Data_Frame.drop(36)
Data_Frame['id']=Data_Frame['State/UT'].apply(lambda x: state_id_map[x])

C:\Users\KARTIK~1\AppData\Local\Temp/ipykernel_32080/2123980960.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\KARTIK~1\AppData\Local\Temp/ipykernel_32080/2123980960.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [89]:
lanes = ['Single Lane Road Report','Two Lane Road Report','Three Lane Road (without Median) Report','Four Lane Road (with Median) Report']
lanes_val = ['Single Lane','Two Lanes','Three Lanes','Four Lanes']
name= 'ETH-DA'
app = dash.Dash(__name__)

colors = {"background": "#0091D5", "text": "white"}

main_style = {
    'background': '#FFFFFF',
    'font-family': "Arial"
}

title_style = {
    'background': '#5473FF',
    'text': '#FFFFFF'
}

desc_style = {
    'background': '#FFFFFF',
    'text': '5473FF'
}
dropdown_style = {
    'background': '#DDE5ED',
    'text': '#221C35'
}

app.layout = html.Div(style={
        'backgroundColor': '#0091D5',
        'font-family': main_style['font-family']
    },children=[   
    html.H1(children="Accidents Statistics Analysis Using Data Visualisation", style={
            'backgroundColor': '#1c4e80',
            'color': title_style['text'],
            'textAlign': 'center','padding':'20px 10px'}   
        ),
        html.Div(children='This dashboard allows user to see the impact of Road Infrastructre on Casualty across the Country differentiated by State',
        style={
            'backgroundColor': '#4cb5f5',
            'color': desc_style['text'],
            'textAlign': 'center','padding':'10px 5px'}),
    html.Div([

        html.Div(style={"width": "50%","font-size":"large",'padding':'10px 5px'},children=[html.Label("Lane Report"),
            dcc.Dropdown(
            id="dropdown",style={"font-size":"large",'color':'#1c4e80'},
            options=[{"label": x, "value": x} for x in lanes],
            value=lanes[0],
            clearable=False,
        )]),

        html.Div([
            dcc.Graph(id="bar-chart",style={'width': '700px', 'height': '120vh'}),
            html.Div([
                dcc.Graph(id="my-map",style={'width': '540px', 'height': '365px'}),
                dcc.Graph(id="my-map2",style={'width': '540px', 'height': '365px'})
            ],style={"display":"flex", "flex-direction":"column","border-left":"dashed 3px",'backgroundColor': '#0091D5'}),
        ], style={"display":"flex", "column-gap":"1px",'backgroundColor': '#0091D5'}),

    ], style={"display":"flex","flex-direction":"column", "gap":"1px",'backgroundColor': '#0091D5'}),
 html.H5("Group Project by: Kartikay, Dhruv and Maitrey",style={'backgroundColor': '#1c4e80','color': 'white','margin':'0','marginTop':'0px','padding':'10px'})
])

@app.callback(
    [Output("bar-chart", "figure"), Output("my-map","figure"), Output("my-map2","figure")],    
    [Input("dropdown", "value")])
def update_bar_chart(lane):
    if(lane=='Single Lane Road Report'):
        fig1 = px.bar(Data_Frame,y='State/UT',x='Single Lane - Accident - 2014', text='Single Lane - Accident - 2014', title=f"Total accidents occured on {lanes_val[0]} in 2014 (states-wise)", width=700)
        fig1.update_traces(texttemplate='%{text:.2s}', textposition='inside')
        fig1.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',plot_bgcolor='white',
        paper_bgcolor='#7EF2F5')
        
        fig2 = go.Figure(data=go.Choropleth(
            geojson=india_states,
            locations=Data_Frame['id'], 
            z = Data_Frame['Single Lane - Killed - 2014'], 
            colorscale="sunsetdark",
            colorbar_title = "Range",
            autocolorscale=False,
            text=Data_Frame['State/UT'], # hover text
            marker_line_color='white', 
            marker_line_width = 0.7,          
        ))
        fig2.update_layout(
            title_text = f'Number of Deaths on {lanes_val[0]} -2014 \n(states-wise)',
            geo_scope='asia',plot_bgcolor='white',
        paper_bgcolor='#7EF2F5'
        )
        fig2.update_geos(
            fitbounds="locations", visible=False
        )

        fig3 = go.Figure(data=go.Choropleth(
            geojson=india_states,
            locations=Data_Frame['id'],
            z = Data_Frame['Single Lane - Injured - 2014'],
            colorscale="Reds",
            colorbar_title = "Range",
            autocolorscale=False,
            text=Data_Frame['State/UT'],
            marker_line_color='white',
            marker_line_width = 0.7
        ))
        fig3.update_layout(
            title_text = f'Number of Casualties on {lanes_val[0]} -2014 (states-wise)',
            geo_scope='asia',
            geo = dict(        
            showlakes=True,
            lakecolor='rgb(225, 225, 225)'),plot_bgcolor='white',
        paper_bgcolor='#7EF2F5'
        )
        fig3.update_geos(fitbounds="locations", visible=False)


        
    if(lane=='Two Lane Road Report'):
        fig1 = px.bar(Data_Frame,y='State/UT',x='Two Lanes - Accident - 2014', text='Two Lanes - Accident - 2014', title=f"Total accidents occured on {lanes_val[1]} in 2014 (states-wise)", width=700) #
        fig1.update_traces(texttemplate='%{text:.2s}', textposition='inside')
        fig1.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',plot_bgcolor='white',
        paper_bgcolor='#7EF2F5')
        
        fig2 = go.Figure(data=go.Choropleth(
            geojson=india_states,
            locations=Data_Frame['id'], 
            z = Data_Frame['Two Lanes - Killed - 2014'],        ##
            colorscale="sunsetdark",
            colorbar_title = "Range",
            text=Data_Frame['State/UT'],
            marker_line_color='white', 
            marker_line_width = 0.7, 
            
        ))
        fig2.update_layout(
            title_text = f'Number of Deaths on {lanes_val[1]} -2014 (states-wise)',
            geo_scope='asia',plot_bgcolor='white',
        paper_bgcolor='#7EF2F5'
        )
        fig2.update_geos(
            fitbounds="locations", visible=False
        )

        fig3 = go.Figure(data=go.Choropleth(
            geojson=india_states,
            locations=Data_Frame['id'],
            z = Data_Frame['Two Lanes - Injured - 2014'],           ##
            colorscale = 'Reds',
            colorbar_title = "Range",
            text=Data_Frame['State/UT'],
            marker_line_color='white', 
            marker_line_width = 0.7, 
        ))
        fig3.update_layout(
            title_text = f'Number of Casualties on {lanes_val[1]} -2014 (states-wise)',
            geo_scope='asia',plot_bgcolor='white',
        paper_bgcolor='#7EF2F5'
        )
        fig3.update_geos(
            fitbounds="locations", visible=False
        )       


    if(lane=='Three Lane Road (without Median) Report'):
        fig1 = px.bar(Data_Frame,y='State/UT',x='3 Lanes or more w.o Median - Accident - 2014', 
        text='3 Lanes or more w.o Median - Accident - 2014', title=f"Total accidents occured on {lanes_val[2]} in 2014 (states-wise)", width=700) ##
        fig1.update_traces(texttemplate='%{text:.2s}', textposition='inside')
        fig1.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',plot_bgcolor='white',
        paper_bgcolor='#7EF2F5')
        
        fig2 = go.Figure(data=go.Choropleth(
            geojson=india_states,
            locations=Data_Frame['id'], 
            z = Data_Frame['3 Lanes or more w.o Median - Killed - 2014'],        ##
            colorscale="sunsetdark",
            colorbar_title = "Range",
            text=Data_Frame['State/UT'],
            marker_line_color='white', 
            marker_line_width = 0.7, 
            
        ))
        fig2.update_layout(
            title_text = f'Number of Deaths on {lanes_val[2]} -2014 (states-wise)',
            geo_scope='asia',plot_bgcolor='white',
        paper_bgcolor='#7EF2F5'
        )
        fig2.update_geos(
            fitbounds="locations", visible=False
        )

        fig3 = go.Figure(data=go.Choropleth(
            geojson=india_states,
            locations=Data_Frame['id'],
            z = Data_Frame['3 Lanes or more w.o Median - Injured - 2014'],           ##
            colorscale = 'Reds',
            colorbar_title = "Range",
            text=Data_Frame['State/UT'],
            marker_line_color='white', 
            marker_line_width = 0.7, 
        ))
        fig3.update_layout(
            title_text = f'Number of Casualties on {lanes_val[2]} -2014 (states-wise)',
            geo_scope='asia',plot_bgcolor='white',
        paper_bgcolor='#7EF2F5'
        )
        fig3.update_geos(
            fitbounds="locations", visible=False
        )

    if(lane=='Four Lane Road (with Median) Report'):
        fig1 = px.bar(Data_Frame,y='State/UT',x='4 Lanes with Median - Accident - 2014', 
        text='4 Lanes with Median - Accident - 2014', title=f"Total accidents occured on {lanes_val[3]} in 2014 (states-wise)", width=700) ##
        fig1.update_traces(texttemplate='%{text:.2s}', textposition='inside')
        fig1.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',plot_bgcolor='white',
        paper_bgcolor='#7EF2F5')
        
        fig2 = go.Figure(data=go.Choropleth(
            geojson=india_states,
            locations=Data_Frame['id'], 
            z = Data_Frame['4 Lanes with Median - Killed - 2014'],        ##
            colorscale="sunsetdark",
            colorbar_title = "Range",
            text=Data_Frame['State/UT'],
            marker_line_color='white', 
            marker_line_width = 0.7,             
        ))
        fig2.update_layout(
            title_text = f'Number of Deaths on {lanes_val[3]} -2014 (states-wise)',
            geo_scope='asia',plot_bgcolor='white',
        paper_bgcolor='#7EF2F5'
        )
        fig2.update_geos(
            fitbounds="locations", visible=False
        )

        fig3 = go.Figure(data=go.Choropleth(
            geojson=india_states,
            locations=Data_Frame['id'],
            z = Data_Frame['4 Lanes with Median - Injured - 2014'],           ##
            colorscale = 'Reds',
            colorbar_title = "Range",
            text=Data_Frame['State/UT'],
            marker_line_color='white', 
            marker_line_width = 0.7, 
        ))
        fig3.update_layout(
            title_text = f'Number of Casualties on {lanes_val[3]} -2014 (states-wise)',
            geo_scope='asia',plot_bgcolor='white',
        paper_bgcolor='#7EF2F5'
        )
        fig3.update_geos(
            fitbounds="locations", visible=False
        )    

    return fig1,fig2,fig3


app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run